By: ***David Santiago Flórez Alsina***

# Homework 2: SemiEdgeList, Triangulation and Art Gallery Problem

-------

1. Write down a code able to identify if the points are ordered in a counterclockwise order. The code should inverse the order of points so that they are counterclockwise ordered if they are in the opposite order. ***Explain and implement your procedure***:

The solution can be found under the `_fix_orientation` method from the SemiEdgeList class, which is described in detail [below](#semiedgelist-class), for now let's take a general overview of the solution.

**Method `_fix_orientation` Steps:**

1. **Identifying the Leftmost Point**:
   - The method starts by identifying the leftmost point within the given list of points, which is a crucial reference for determining the orientation of the polygon, this is because the leftmost point is always part of the polygon's convex hull, and by taking a point which is part of the convex hull, we can ensure that the polygon's orientation is consistent and doesn't depend on local concavities, rotations, or other factors.

2. **Obtaining Index of Leftmost Point in the list of points, and with this get the next and previous points**:
   - The index of this leftmost point is recorded for future reference.
   - This is achieved by considering the points cyclically, ensuring the indices wrap around the list.

4. **Calculating Turn**:
   - With the leftmost point, previous point, and next point determined, the method computes the turn using the cross product. This turn calculation indicates whether the polygon is oriented clockwise or counterclockwise.

5. **Adjusting Point Order**:
   - If the computed turn indicates a clockwise orientation, it implies that the points are arranged in a clockwise order. To correct this, the method reverses the order of the points, effectively converting a clockwise orientation into a counterclockwise one.



-------
2. Write down a code able to read the ordered points and create a doubly-connected edge list for the simple polygon. ***Print the doubly-connected edge list related to the simple polygon.***    

The solution can be found under the `SemiEdgeList` class, which is described in detail [below](#semiedgelist-class).
The desired print can be found implemented under the 'show_data_structure' method, the output can be seen in [this section](#semiedgelist-print).

-------

3. Implement an algorithm able to split the given polygon into y-monotone polygons. Give your answer in terms of doubly-connected edge lists. Plot the polygon split into y-monotone polygons.

Now we import the base dependencies needed for the Homework:

In [8]:
#external dependencies
import pandas as pd
import numpy as np 
from copy import deepcopy
from typing import List, Tuple, Literal, Union

## Vector Class

The `Vector` class manages computational geometry vectors, offering operations like slope calculation, distance measurement, and angle determination. It includes methods for finding extreme points and supports common vector arithmetic. The class employs lexicographical ordering for comparisons, enhancing its versatility in geometric computations.

In [9]:
class Vector:
    TURN_LEFT = 1
    TURN_RIGHT = -1
    TURN_NONE = 0

    CLOCKWISE_TURN = TURN_RIGHT
    ANTICLOCKWISE_TURN = TURN_LEFT


    def __init__(self, vector: np.ndarray):
        
        #it's a column vector
        if (vector.shape[0] > 1) and (vector.shape[1] == 1):
            self.vector = vector
            self.T = vector.T
            self.ndimensions = self.vector.shape[0]
        
        #it's a row vector
        elif (vector.shape[1] > 1) and (vector.shape[0] == 1):
            self.vector = vector.T
            self.T = vector.T
            self.ndimensions = self.vector.shape[0]
        else:
            raise TypeError(f"Should be a row or a column vector, yet has shape: {vector.shape}")

    #================================================
    #            Some important operations
    #================================================

    def calculate_slope(self, vector2: 'Vector') -> float:
        """
            Calculates the slope between self and vector2.

            Input:
                vector2: Vector
            Output:
                slope: float
        """

        if self[0] == vector2[0]:
            return np.inf
        return (self[1] - vector2[1]) / (self[0] - vector2[0])


    def calculate_distance(self, vector2: 'Vector') -> float:
        """
            Calculates the distance between self and vector2.

            Input:
                vector2: Vector
            Output:
                distance: float
        """
        return np.linalg.norm(self.vector - vector2.vector)
    
    def get_furthest_point(self, points: List['Vector']) -> 'Vector':

        """
            Finds the furthest point from self.

            Input:
                points: Vector or List[Vector]
            Output:
                furthest_point: Vector
        """

        # calculate the distance from v0 to each point
        distances = [self.calculate_distance(point) for i, point in enumerate(points)]
        
        # find the maximum distance
        maxdist = max(distances)

        # find the indexes of the points with the maximum distance
        indexes = [i for i, d in enumerate(distances) if d == maxdist]

        return points[indexes[0]]


    @staticmethod
    def calculate_angle(A:'Vector', B:'Vector', C:'Vector') -> Tuple[float, float]:
        """
            Calculates the angle between the vectors AB and BC. This is the same
            as the angle between a walk from A to B and a walk from B to C, and 
            get the angle at the bend.

            Args:
            -----------
                A: Vector
                B: Vector
                C: Vector
            
            Returns:
            -----------
                angle_rad: float
                    Angle in radians
                angle_deg: float
                    Angle in degrees
        """

        AB = B - A
        BC = B - C

        # calculate the cross product between A and B (which are column vectors)
        cross_product = np.cross(AB.vector.T, BC.vector.T)

        # calculate the dot product between A and B (which are column vectors)
        dot_product = np.dot(AB.vector.T, BC.vector)

        # get direction, which is the sign of the cross product and determine if 
        # the angle is convex or concave
        direction = np.sign(cross_product)

        # calculate the angle between A and B, using the cross and dot products
        # with the arctan2 function
        angle_rad = np.arctan2(cross_product, dot_product) + np.pi*direction

        # convert the angle to degrees
        angle_deg = np.degrees(angle_rad)

        #make sure both angles are positive with modular arithmetic
        angle_rad = (angle_rad + 2) % (2)
        angle_deg = (angle_deg + 360) % 360

        return float(angle_rad), float(angle_deg)


    @staticmethod
    def calculate_turn(vector1: 'Vector', vector2: 'Vector', vector3: 'Vector') -> int:

        """
            Calculates if the walk from vector1 to vector2 to vector3 is a left turn
            or a right turn.

            Input:
                vector1: Vector
                vector2: Vector
                vector3: Vector
            Output:
                1: if the walk is a left turn
                -1: if the walk is a right turn
                0: if the points are collinear
        """

        # calculate the cross product between A and B (which are column vectors)
        cross_product = np.cross((vector2 - vector1).vector.T, (vector3 - vector2).vector.T)

        # get direction, which is the sign of the cross product and determine if 
        # the angle is convex or concave
        direction = np.sign(cross_product)

        return int(direction)

    #================================================
    #               Static methods
    #================================================
    @staticmethod
    def get_leftmost_point(points: List['Vector']) -> 'Vector':
        """
            Finds the leftmost point from a list of points.

            Input:
                points: List[Vector]
            Output:
                leftmost_point: Vector
        """
        # find the leftmost point
        leftmost_point = points[0]
        for point in points[1:]:
            if point[0] < leftmost_point[0]:
                leftmost_point = point
        return leftmost_point
    
    @staticmethod
    def get_rightmost_point(points: List['Vector']) -> 'Vector':
        """
            Finds the rightmost point from a list of points.

            Input:
                points: List[Vector]
            Output:
                rightmost_point: Vector
        """
        # find the rightmost point
        rightmost_point = points[0]
        for point in points[1:]:
            if point[0] > rightmost_point[0]:
                rightmost_point = point
        return rightmost_point

    @staticmethod
    def build_random_vectors(nvectors:int, minval = -10, maxval = 10) -> List['Vector']:
        vectors = []
        for i in range(nvectors):
            vectors.append(Vector(np.random.randint(minval, maxval, (2, 1))))

        return vectors
    
    @staticmethod
    def cast_to_vector(*vectors: np.ndarray) -> List['Vector']:
        """
            Casts a list of numpy arrays to a list of column vectors.
        """
        #reshape the vectors to be column vectors
        vectors = [vector.reshape((2, 1)) for vector in vectors]
        return [Vector(vector) for vector in vectors]

    #================================================
    #               Operator overloading
    #================================================
    
    def times_scalar(self, scalar: float) -> 'Vector':
        return Vector(self.vector * scalar)

    def __add__(self, vector2: 'Vector') -> 'Vector':
        return Vector(self.vector + vector2.vector)
    
    def __sub__(self, vector2: 'Vector') -> 'Vector':
        return Vector(self.vector - vector2.vector)
    
    def __mul__(self, other: 'Vector') -> 'Vector':
        if isinstance(other, Vector): 
            return Vector(self.vector * other.vector)
        else: 
            return NotImplemented

    def __rmul__(self, other):
        return self.__mul__(other) 
    
    def __truediv__(self, number: float) -> 'Vector':
        return Vector(self.vector / number)

    def __getitem__(self, index: int) -> float:
        return float(self.vector[index][0])

    def __repr__(self):
        return f"Vec({self.vector[0][0]}, {self.vector[1][0]})"

    def __str__(self):
        return f"Vec({self.vector[0][0]}, {self.vector[1][0]})"

    #================================================
    #              Order relations overloading
    #================================================

    def __eq__(self, vector: 'Vector') -> bool:
        return (self[0] == vector[0]) and (self[1] == vector[1])

    def __ne__(self, vector: 'Vector') -> bool:
        return (self[0] != vector[0]) or (self[1] != vector[1])

    def __lt__(self, vector2: 'Vector') -> bool:
        return (self[1] > vector2[1]) or (self[1] == vector2[1] and self[0] < vector2[0])

    def __gt__(self, vector2: 'Vector') -> bool:
        return (self[1] < vector2[1]) or (self[1] == vector2[1] and self[0] > vector2[0])
    
    def __ge__(self, vector2: 'Vector') -> bool:
        return (self > vector2) or (self == vector2)
    
    def __le__(self, vector2: 'Vector') -> bool:
        return (self < vector2) or (self == vector2)
    
    def __hash__(self):
        return hash((self[0], self[1]))

## Segment class 

The `Segment` class in computational geometry represents 2D line segments. It defines operations like calculating slope, distance, and midpoint. Methods determine if a point lies on the segment and the rotation direction of a given point. The class also checks for intersections between segments. It offers detailed intersection detection, including endpoints and lines formed by segments. Additionally, it provides a static method to generate random segments. Comparison methods ensure equality checks. 

In [10]:
class Segment():

    def __init__(self, start: Vector, end: Vector, sort_ends = False) -> None:
        """
            This class is used to represent a segment in 2D space.
            start and end are the vertices of the segment. Starting from start, and ending in end.
        """
        self.start = start
        self.end = end

        #make sure that the start is the lowest leftmost point, 
        #refer to our order operator overload in vector.py
        if sort_ends and (self.start > self.end):
            self.start, self.end = self.end, self.start


    def get_midpoint(self) -> Vector:
        """
            Calculates the midpoint of the segment.
        """
        return (self.start + self.end)/2

    def calcutale_slope(self) -> float:
        """
            Calculates the slope of segment.
        """
        deltas = self.end - self.start

        if deltas[0] == 0:
            return np.inf

        return (deltas[1])/(deltas[0])
        
    def calculate_distance(self) -> float:
        """
            Calculates the distance between end and v2.
        """
        return np.linalg.norm(self.end - self.start)

    
    def on_segment(self, v3: Vector) -> bool:
        """
            Checks if v3 is on the segment startend area,
            this area is defined by the rectangle with vertices start and end.
        """
        inx = min(self.start[0], self.end[0]) <= v3[0] <= max(self.start[0], self.end[0]) 
        iny = min(self.start[1], self.end[1]) <= v3[1] <= max(self.start[1], self.end[1])

        if (inx and iny):
            return True
        return False

    def direction(self, vector3: Vector) -> int:
        """
            Given our segment, and a vector3, determine if vector3 is rotated clockwise or anticlockwise, 
            or if they are colineal, with respect to the segment.

            Output:
                1: if the walk is a left turn
                -1: if the walk is a right turn
                0: if the points are collinear
        """
        det = np.linalg.det(np.column_stack([(vector3 - self.start).vector, (self.end - self.start).vector]))
        return np.sign(det)

    def segments_intersect(self, other_segment: 'Segment') -> bool:

        """
            Given one segment, determine if the segments they form intersect.
            Segments are startend from self, and startend from other_segment.
        """

        #directions with respect to other_segment
        dir1 = other_segment.direction(self.start)
        dir2 = other_segment.direction(self.end)
        
        #directions with respect to self
        dir3 = self.direction(other_segment.start)
        dir4 = self.direction(other_segment.end)

        #well behaved case
        if (dir1*dir2 < 0) and (dir3*dir4 < 0):
            return True
    
        #collinear case with respect to other_segment
        elif (dir1 == 0) and (other_segment.on_segment(self.start)):
            return True

        elif (dir2 == 0) and (other_segment.on_segment(self.end)):
            return True

        #collinear case with respect to self
        elif (dir3 == 0) and (self.on_segment(self.start)):
            return True

        elif (dir4 == 0) and (self.on_segment(self.end)):
            return True
        
        else:
            return False
    
    def get_intersection_of_segments_general(self, other_segment: 'Segment') -> Union['Segment', Vector]:

        """
            This method uses the find_intersection_on_endpoints, 
            the find_intersection, and the find_interval_intersection methods,
            which respectively get the intersect on endpoints (if any),
            the intersection between segments (if any), 
            and the intersection on an interval (if any).
        """
    
        # Find the intersection point (if any)
        intersection = self.find_intersection(other_segment)

        # Find the intersection on endpoints (if any)
        intersection_on_endpoints = self.find_intersection_on_endpoints(other_segment)

        # Find the intersection on segments (if any)
        intersection_on_segments = self.find_interval_intersection(other_segment)

        if intersection_on_segments is not None:
            return intersection_on_segments
        elif intersection_on_endpoints is not None:
            return intersection_on_endpoints
        else:
            return intersection


    
    def find_intersection_on_endpoints(self, other_segment: 'Segment') -> Union['Segment', None]:
        """
            Finds the intersection of the segments on the endpoints.
        """
        if (self.start == other_segment.start) or (self.start == other_segment.end):
            return self.start
        elif (self.end == other_segment.start) or (self.end == other_segment.end):
            return self.end
        else:
            return None
    
    def find_intersection(self, other_segment: 'Segment') -> Vector:
        """
            Finds the intersection of the lines formed by the segments endv2 and v3v4.
        """

        difference1 = (self.start - self.end).vector
        difference2 = (other_segment.start - other_segment.end).vector
        result = (other_segment.start - self.end).vector

        matrix = np.column_stack([difference1, difference2])
        
        #if the matrix is singular, then the lines are parallel, 
        # but they might intersect on the endpoints
        try:
            alpha_beta = np.linalg.solve(matrix, result).reshape(2,)

        except np.linalg.LinAlgError as e:
            if 'Singular matrix' in str(e):
                return self.find_intersection_on_endpoints(other_segment)
            else: 
                raise e
            
        intersect = self.start.times_scalar(alpha_beta[0]) + self.end.times_scalar(float(1 - alpha_beta[0]))

        return intersect
    
    def find_interval_intersection(self, other_segment: 'Segment') -> Union['Segment', None]:
        """
            Finds the interval intersection of the segments.
            If there is no intersection, return None.
        """

        #check if the segments have the same slope
        same_slope = self.calcutale_slope() == other_segment.calcutale_slope()

        # given that they have the same slope, check one of the 
        # segments is contained in the other, we do this by taking the endpoints of the segments
        # and checking if they are contained in the other segment
        p0_on_segment = other_segment.on_segment(self.start)
        p1_on_segment = other_segment.on_segment(self.end)
        p2_on_segment = self.on_segment(other_segment.start)
        p3_on_segment = self.on_segment(other_segment.end)
        ans = None

        if same_slope:
            if (p0_on_segment and p1_on_segment):
                ans =  Segment(self.start, self.end)
            elif (p2_on_segment and p3_on_segment):
                ans = Segment(other_segment.start, other_segment.end)
            elif (p0_on_segment and p3_on_segment):
                ans = Segment(self.start, other_segment.end)
            elif (p2_on_segment and p1_on_segment):
                ans = Segment(other_segment.start, self.end)
            else:
                return ans
        else:
            return ans

        #there could be a posibility that it matches as a segment
        #something that it's indeed a point
        if ans.start == ans.end:
            return None 
        else:
            return ans

    @staticmethod
    def build_random_segments(nsegments: int=10)-> List['Segment']:
        """
            Builds nsegments random segments.
        """
        segments = []

        for _ in range(nsegments):
            start = Vector.build_random_vectors(1)[0]
            end = Vector.build_random_vectors(1)[0]
            segments.append(Segment(start, end))

        return segments

    def __repr__(self) -> str:
        return f"Segment({self.start}, {self.end})"
    
    def __eq__(self, other_segment: 'Segment') -> bool:

        if (self is not None and other_segment is None) or (self is None and other_segment is not None):
            return False

        self_set = set([self.start, self.end])
        other_set = set([other_segment.start, other_segment.end])
        return self_set == other_set
        
    def __ne__(self, other_segment: 'Segment') -> bool:
        return not self.__eq__(other_segment)
    
    def __hash__(self):
        return hash((self.start, self.end))


## GeometricNode class

The `GeometricNode` class represents nodes in computational geometry with attributes like a 2D point, a name, and an incident edge. It offers convenient access to coordinates, hashing based on the point, and lexicographical ordering capabilities. 


In [11]:

class GeometricNode():
    """
        This class represents a node in a double connected edge list.
    """

    def __init__(self,
                 point: Vector,
                 name: str = None, 
                 incident_edge: Segment = None,):

        """
            Every node name is a string that represents the name of the node, and
            should have a format like this: 'N1', 'N2', 'N3', etc.
        """
        self.name = name
        self.value = None
        self.point = point
        self.incident_edge = incident_edge

    @property
    def x(self) -> float:
        return float(self.point[0])
    
    @property
    def y(self) -> float:
        return float(self.point[1])

    def __hash__(self) -> int:
        return hash(self.point)

    def __getitem__(self, index: int) -> float:
        return float(self.point[index])
    
    def __repr__(self) -> str:
        return f"{self.name} {self.point}"
    
    def __lt__(self, other_node: 'GeometricNode') -> bool:
        return self.point < other_node.point

    def __le__(self, other_node: 'GeometricNode') -> bool:
        return self.point <= other_node.point
    
    def __gt__(self, other_node: 'GeometricNode') -> bool:
        return self.point > other_node.point

    def __ge__(self, other_node: 'GeometricNode') -> bool:
        return self.point >= other_node.point

    def __eq__(self, other_node: 'GeometricNode') -> bool:
        return self.point == other_node.point
    
    def __ne__(self, other_node: 'GeometricNode') -> bool:
        return self.point != other_node.point


## Face class

The `Face` class in computational geometry defines polygon faces, distinguishing between interior and exterior types. It holds a list of semi-edges, a name, and a face type. Methods allow setting semi-edges, adding new ones, and determining face types based on edge turns. 

In [12]:
class Face():
    
    INTERIOR_FACE = 1
    EXTERIOR_FACE = 2 

    def __init__(self, name: str = None):

        self.semi_edges: List['SemiEdge'] = []
        self._name: str = name
        self.face_type: int = -1
    
    @property
    def name(self) -> str:
        return self._name
    
    def set_name(self, name: str) -> None:
        self._name = name

    def set_semi_edges(self, semi_edges: List['SemiEdge']) -> None:
        self.semi_edges = semi_edges
        self.set_face_type()

    def add_semi_edge(self, semi_edge: 'SemiEdge') -> None:
        self.semi_edges.append(semi_edge)
    
    def set_face_type(self):
        
        #get the leftmost point among the set of points in the polygon
        points = [semiedge.origin.point for semiedge in self.semi_edges]
        left_most_point = Vector.get_leftmost_point(points)
        
        #get the index corresponding to the segment that has that point as 
        #orgin
        idx = points.index(left_most_point)
        left_most_semiedge = self.semi_edges[idx]

        #get the origin point, the previous and the next of these
        origin = left_most_semiedge.start
        prev = left_most_semiedge.prev_edge.start
        next_ = left_most_semiedge.end

        #get the way the face is turning 
        turn = Vector.calculate_turn(prev, origin, next_)

        if turn == 1:
            self.face_type = Face.EXTERIOR_FACE
        elif turn == -1:
            self.face_type = Face.INTERIOR_FACE

    def __str__(self) -> str:
        return f"{self._name}"
    
    def __repr__(self) -> str:
        return f"{self._name}"


## SemiEdge class

The `SemiEdge` class in computational geometry represents directed edges within a planar subdivision. It holds attributes like origin, next vertex, incident face, and associated segment. Methods allow setting cycle relationships and helper vectors for triangulation. 

In [13]:
class SemiEdge():

    def __init__ (self, 
                  origin: Union[GeometricNode, Vector],
                  next_: Union[GeometricNode, Vector],
                  incident_face: Face = None, 
                  need_cast: bool = False):


        if need_cast:
            self.origin: GeometricNode = GeometricNode(origin)
            self.next_: GeometricNode = GeometricNode(next_)

        else:
            self.origin: GeometricNode = origin
            self.next_: GeometricNode = next_

        self.seg: Segment = Segment(self.origin.point, self.next_.point)
        self.incident_face: Face = incident_face

        #completely twisted version of self
        self.next_edge: SemiEdge = None
        self.prev_edge: SemiEdge = None
        self.twin: SemiEdge = None
        
        #name of the semiedge
        self.name: str = None
        
        # This is used for the triangulation, the literal part is a constant 
        # that indicates the helper vector type, this could be START_VERTEX,
        # END_VERTEX, SPLIT_VERTEX, MERGE_VERTEX, REGULAR_VERTEX
        self.helper: Tuple[Vector, Literal]= None
    
    @property
    def start(self) -> Vector:
        return self.seg.start
    
    @property
    def end(self) -> Vector:
        return self.seg.end

    def set_next_edge(self, next_edge: 'SemiEdge') -> None:
        self.next_edge = next_edge
    
    def set_prev_edge(self, prev_edge: 'SemiEdge') -> None:
        self.prev_edge = prev_edge
    
    def set_helper(self, helper: Tuple[Vector, int]) -> None:
        self.helper = helper

    def set_twin(self, twin: 'SemiEdge') -> None:
        self.twin = twin
    
    def set_name(self, name: str) -> None:
        self.name = name

    def __repr__(self) -> str:
        if self.name != None:
            return f"{self.name}"
        else: 
            return f"SE({self.origin}, {self.next_})"
    
    def __str__(self) -> str:
        if self.name != None:
            return f"{self.name}"
        else: 
            return f"SE({self.origin}, {self.next_})"
    
    def __eq__(self, semiedge: 'SemiEdge') -> bool:
        return self.seg == semiedge.seg 
        
    def __ne__(self, semiedge: 'SemiEdge') -> bool:
        return self.seg != semiedge.seg

    def __hash__(self) -> int:
        return hash(self.seg)


## SemiEdgeList class

The `SemiEdgeList` class manages a planar subdivision's structure, composed of geometric nodes and semi-edges. It handles orientation adjustments and creates nodes from a list of points. Semi-edges connect these nodes, forming a polygon. The class can add new semi-edges while maintaining topological relationships. It also tracks incident edges of vertices and provides methods for retrieving the faces of the subdivision. 

In [14]:
class SemiEdgeList():

    def __init__(self, list_of_points: List[Vector], name: str):
        
        self.list_of_nodes : List[GeometricNode] = []
        self.semi_edges : List[SemiEdge] = []
        self.faces : List[Face] = []
        self.name = name
        
        self._fix_orientation(list_of_points)
        self._build_nodes(list_of_points)
        self._build_semi_edges()
    
    def _fix_orientation(self, list_of_points: List[Vector]) -> None:
        """
            This method reverses the orientation of the polygon if it's clockwise.
            Otherwise, it does nothing.
        """

        #get the leftmost point of the polygon which is one of the points of the
        #convex hull, by looking at the prev and next points of the leftmost point
        #and then computing the turn of the three points (prev, leftmost, next)
        #we can determine if the polygon is clockwise or counterclockwise
        envelope_component = Vector.get_leftmost_point(list_of_points)

        #get the index of the leftmost point
        index = list_of_points.index(envelope_component)

        #get the prev and next points of the leftmost point
        prev_point = list_of_points[(index-1)%len(list_of_points)]
        next_point = list_of_points[(index+1)%len(list_of_points)]

        #get the turn of the three points
        turn = Vector.calculate_turn(prev_point, envelope_component, next_point)

        if turn == Vector.CLOCKWISE_TURN:
            list_of_points.reverse()

    def show_data_structure(self) -> pd.DataFrame:
        df = pd.DataFrame(columns = ["name", "origin", "next", "prev_edge", "next_edge", "twin", "prev_edge_twin", "next_edge_twin", "incident_face"])

        for semiedge in self.semi_edges:
            line = pd.DataFrame([{"name": semiedge.name,
                                  "origin": semiedge.origin.name, 
                                  "next": semiedge.next_.name, 
                                  "prev_edge": semiedge.prev_edge, 
                                  "next_edge": semiedge.next_edge,
                                  "twin": semiedge.twin, 
                                  "prev_edge_twin": semiedge.twin.prev_edge,
                                  "next_edge_twin": semiedge.twin.next_edge,
                                  "incident_face": semiedge.incident_face.name},])
            df = pd.concat([df, line])

        df = df.reset_index(drop=True)

        return df

    def add_new_semi_edges(self, semi_edges: List[SemiEdge]) -> None:
        """
            This function adds semi-edges to the list of semi-edges.

            This is a delicate process, because with each semi-edge we add 
            it's necessary to add the next and prev edges of each semi-edge.
            And if this semiedge intersects with another semi-edge, we have to
            add the intersection point to the list of nodes, and add the new
            semi-edges that are created by the intersection.
        """
        
        for semiedge in semi_edges:
            self.add_new_edge(semiedge)


    def _build_nodes(self, list_of_points: List[Vector]) -> None:
        """
            This function builds the nodes from a list of points.
            The name of each node is the index of the point in the list of points.
        """
        self.list_of_nodes = []

        #

        for i in range(len(list_of_points)):
            node = GeometricNode(point = list_of_points[i], name = f"{self.name}N{i}")

            #avoid adding repeated nodes
            if node not in self.list_of_nodes:
                self.list_of_nodes.append(node)
            else:
                raise Exception("Repeated node, watch out!")

    def _build_semi_edges(self) -> List[SemiEdge]:
        """
            This function builds the semi-edges from the list of nodes.
            This assumes that the list of nodes is ordered in a way that
            the next node is the next node in the polygon (this is true just 
            at first build time).

            The function builds the semi-edges, the next and prev edges of each
            semi-edge, and the faces of each semi-edge.
        """
        self.semi_edges = []
        
        # Create the semi-edges from the list of nodes
        for i in range(len(self.list_of_nodes)):
            semi_edge = SemiEdge(origin = self.list_of_nodes[i], 
                                 next_ = self.list_of_nodes[(i+1)%len(self.list_of_nodes)])
            twin = SemiEdge(origin = semi_edge.next_, 
                            next_ = semi_edge.origin)

            #handle twins mutually
            semi_edge.set_twin(twin)
            twin.set_twin(semi_edge)

            #add to the list of semi-edges, and set the name
            semi_edge.set_name(f"SE{len(self.semi_edges)}")
            twin.set_name(f"SE{len(self.semi_edges)}''")
            self.semi_edges.append(semi_edge)

        # Set the next and prev edges of each semi-edge
        for i in range(len(self.semi_edges)):
            self.semi_edges[i].set_next_edge(self.semi_edges[(i+1)%len(self.semi_edges)])
            self.semi_edges[i].set_prev_edge(self.semi_edges[(i-1)%len(self.semi_edges)])

        # Set the next and prev edges of each twin semi-edge
        for i in range(len(self.semi_edges)):
            self.semi_edges[i].twin.set_next_edge(self.semi_edges[(i-1)%len(self.semi_edges)].twin) 
            self.semi_edges[i].twin.set_prev_edge(self.semi_edges[(i+1)%len(self.semi_edges)].twin)
        
        # add faces to the semi-edges
        self._set_faces(self.semi_edges)
        self._set_faces(self.semi_edges, twins=True)

        return self.semi_edges

    def _reset_faces(self):
        """
            This function resets the faces of the polygon.
            This is necessary because when we add a new edge, we have to
            reset the faces of the polygon, because the faces are not correct
            anymore.
        """
        self.faces = []

        for s in self.semi_edges:
            #remove the face from memory
            del s.incident_face
            del s.twin.incident_face

            #remove the face from attributes
            s.incident_face = None
            s.twin.incident_face = None
        
        self._set_faces(self.semi_edges)
        self._set_faces(self.semi_edges, twins=True)

    def _set_faces(self, 
                   edges: List[SemiEdge], 
                   twins: bool = False) -> None:
        """
            This function adds the faces to the semi-edges.

            The idea is to take a semi-edge, and iter to the next semi-edge until
            we reach the initial semi-edge. All the semi-edges that we visited until 
            reaching the initial semi-edge form a face.

            Input:
            --------
                edges: List[SemiEdge]
                    The list of semi-edges to which we want to add the faces
                
                twins: bool
                    If True, then it's referring to add the faces to the twins of the
                    semi-edges. If False, then it's referring to add the faces to the
                    semi-edges.
        """

        faces_count = 0

        # We iterate over the semi-edges
        for semi_edge in edges:

            if twins:
                semi_edge = semi_edge.twin

            #print(f"iterating over semi-edge {semi_edge}, face: {semi_edge.incident_face}, face_count: {faces_count}")
            # If the semi-edge doesn't have a face, then we add a face
            if semi_edge.incident_face == None:
                
                # We create a new face
                if twins:
                    face_name = f"{semi_edge.twin.name}''" #name for twin face
                else:
                    face_name = f"{self.name}:F{faces_count}" #name for face

                face = Face(name = face_name)

                # We iterate over the next semi-edge
                next_semi_edge = semi_edge.next_edge

                # We set the face to the semi-edge
                semi_edge.incident_face = face
                next_semi_edge.incident_face = face

                # We add the semi-edge to the face
                face.add_semi_edge(semi_edge)
                face.add_semi_edge(next_semi_edge)

                # While the next semi-edge is not the initial semi-edge
                while next_semi_edge != semi_edge:
                    #print(f"\t next_semi_edge: {next_semi_edge}, semi_edge: {semi_edge}, face: {face}, face_count: {faces_count}")

                    # We iterate over the next semi-edge
                    next_semi_edge = next_semi_edge.next_edge

                    # We add the face to the semi-edge
                    next_semi_edge.incident_face = face

                    # We add the semi-edge to the face
                    face.add_semi_edge(next_semi_edge)

                face.set_face_type()
                faces_count += 1

                if not twins:
                    self.faces.append(face)

    def add_new_edge(self, semiedge: SemiEdge) -> None:
        """
            This function adds a new edge to the list of semi-edges.
            This is a delicate process, because with each semi-edge we add
            it's necessary to add the next and prev edges of each semi-edge.
        """

        #check if the semiedge is already in the list of semiedges
        if semiedge in self.semi_edges:
            print("The semiedge is already in the list of semiedges")
            return

        twin = SemiEdge(origin = semiedge.next_,
                        next_  = semiedge.origin)

        #add the twin and semiedge name 
        semiedge.set_name(f"SE{len(self.semi_edges)}")
        twin.set_name(f"SE{len(self.semi_edges)}''")

        #identify the semiedges with origin in any of the ends 
        #of the semiedge I want to add

        print(f"I want to add {semiedge}, {semiedge.seg}")
        related_edges_orig = self.get_incident_edges_of_vertex(semiedge.origin)
        related_edges_next = self.get_incident_edges_of_vertex(semiedge.next_)

        #from those related edges filter the ones that have the same incident face and 
        #that are exterior frontier 
        related_edges_orig = [e for e in related_edges_orig if e.incident_face.face_type == Face.EXTERIOR_FACE]
        related_edges_next = [e for e in related_edges_next if e.incident_face.face_type == Face.EXTERIOR_FACE]

        related_edges = related_edges_orig + related_edges_next
        found = False
        pair: Tuple[SemiEdge, SemiEdge] = ()
        print(f"RELATED EDGES {related_edges}")

        #search the pair that has the same incident face
        for e1 in related_edges:
            for e2 in related_edges:

                if (e1.incident_face == e2.incident_face) and (e1 != e2):
                    pair = (e1, e2)
                    found = True
                    break
            if found:
                break

        if not found:
            raise Exception("No pair of edges found")
        
        related_a = pair[0] if pair[0].origin == semiedge.origin else pair[1]
        related_b = pair[1] if related_a == pair[0] else pair[0]
        
        #reset prev edge of existent edges in the polygon
        related_a.prev_edge.set_next_edge(semiedge)
        related_b.prev_edge.set_next_edge(twin)

        #set the previous edges of the new semiedges
        semiedge.set_prev_edge( related_a.prev_edge )
        twin.set_prev_edge( related_b.prev_edge )

        #set the next edges of the new semiedges 
        semiedge.set_next_edge( related_b )
        twin.set_next_edge( related_a )

        #reset the prev edge of the existent edges in polygon
        related_b.set_prev_edge(semiedge) 
        related_a.set_prev_edge(twin)   

        semiedge.twin = twin
        self.semi_edges.append(semiedge)
        
        #set origin and next vertex names
        semiedge.origin.name = related_edges_orig[0].origin.name
        semiedge.next_.name = related_edges_next[0].origin.name
        twin.origin.name = related_edges_next[0].origin.name
        twin.next_.name = related_edges_orig[0].origin.name

        #reset the faces of the polygon
        self._reset_faces()

    def get_incident_edges_of_vertex(self, vertex: GeometricNode) -> List[SemiEdge]:
        """
            This function returns the edges that have the vertex as origin.

            Input:
            -------
                vertex: GeometricNode
                    The vertex to which the edges start

            Output:
            -------
                starts_at_vertex: List[SemiEdge]
                    The list of edges that start at the vertex
        """

        starts_at_vertex = []

        for semiedge in self.semi_edges:
            if semiedge.origin == vertex:
                starts_at_vertex.append(semiedge)
            elif semiedge.twin.origin == vertex:
                starts_at_vertex.append(semiedge.twin)

        return starts_at_vertex


    def __getitem__(self, index: int) -> SemiEdge:
        return self.semi_edges[index]

    def __len__(self) -> int:
        return len(self.semi_edges)
    
    def __iter__(self):
        return iter(self.semi_edges)

    def __str__(self) -> str:
        return f"{self.semi_edges}"
    
    def __repr__(self) -> str:
        return f"{self.semi_edges}"

## SemiEdgeList Print

In [16]:
#build points of the polygon
points3 = [np.array([9, -1]), np.array([8, 6]), np.array([7, -3]), np.array([6, 3]),
           np.array([4, -4]), np.array([2, -4]), np.array([3, 4]), np.array([4, 2.5]),
           np.array([5, 12]), np.array([7, 12]), np.array([10, 6]), np.array([9.5, 11]),
           np.array([11, 15]), np.array([13, 15]), np.array([12, 10]), np.array([13, 3]),
           np.array([11, 4]), np.array([10, -1])]

#cast the points to vectors
vectors3 = Vector.cast_to_vector(*points3)

#build the semiedges from the vectors
semiedges = SemiEdgeList(vectors3, name = "S1")
semiedges.show_data_structure()

/tmp/ipykernel_198701/32764546.py:151: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(direction)


,name,origin,next,prev_edge,next_edge,twin,prev_edge_twin,next_edge_twin,incident_face
0,SE0,S1N0,S1N1,SE17,SE1,SE0'',SE1'',SE17'',S1:F0
1,SE1,S1N1,S1N2,SE0,SE2,SE1'',SE2'',SE0'',S1:F0
2,SE2,S1N2,S1N3,SE1,SE3,SE2'',SE3'',SE1'',S1:F0
3,SE3,S1N3,S1N4,SE2,SE4,SE3'',SE4'',SE2'',S1:F0
4,SE4,S1N4,S1N5,SE3,SE5,SE4'',SE5'',SE3'',S1:F0
5,SE5,S1N5,S1N6,SE4,SE6,SE5'',SE6'',SE4'',S1:F0
6,SE6,S1N6,S1N7,SE5,SE7,SE6'',SE7'',SE5'',S1:F0
7,SE7,S1N7,S1N8,SE6,SE8,SE7'',SE8'',SE6'',S1:F0
8,SE8,S1N8,S1N9,SE7,SE9,SE8'',SE9'',SE7'',S1:F0
9,SE9,S1N9,S1N10,SE8,SE10,SE9'',SE10'',SE8'',S1:F0
